In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

TORCH_DTYPE = 'bfloat16'
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=getattr(torch, TORCH_DTYPE)
)

In [2]:
tokenizer = AutoTokenizer.from_pretrained('mesolitica/malaysian-llama2-7b-32k-instructions')

In [3]:
model = AutoModelForCausalLM.from_pretrained(
    'mesolitica/malaysian-llama2-7b-32k-instructions',
    use_flash_attention_2 = True,
    quantization_config = nf4_config
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
def parse_llama_chat(messages):

    system = messages[0]['content']
    user_query = messages[-1]['content']

    users, assistants = [], []
    for q in messages[1:-1]:
        if q['role'] == 'user':
            users.append(q['content'])
        elif q['role'] == 'assistant':
            assistants.append(q['content'])

    texts = [f'<s>[INST] <<SYS>>\n{system}\n<</SYS>>\n\n']
    for u, a in zip(users, assistants):
        texts.append(f'{u.strip()} [/INST] {a.strip()} </s><s>[INST] ')
    texts.append(f'{user_query.strip()} [/INST]')
    prompt = ''.join(texts).strip()
    return prompt

In [5]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'kwsp tu apa'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')

In [6]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
tokenizer.decode(r[0])

'<s> [INST] <<SYS>>\nawak adalah AI yang mampu jawab segala soalan\n<</SYS>>\n\nkwsp tu apa [/INST] Kumpulan Wang Simpanan Pekerja (KWSP) ialah organisasi yang ditubuhkan untuk menggalakkan simpanan dan pelaburan dalam kalangan rakyat Malaysia untuk persaraan mereka. KWSP menyediakan perkhidmatan simpanan dan pelaburan kepada ahlinya, yang termasuk semua pekerja yang bekerja di Malaysia, dan menawarkan beberapa faedah kepada mereka, seperti dividen tahunan. </s>'

In [7]:
messages = [
    {'role': 'system', 'content': 'awak adalah AI yang mampu jawab segala soalan'},
    {'role': 'user', 'content': 'awat malaysia ada jabatan koko, malaysia bukan buat keluaq koko banyak pun'}
]
prompt = parse_llama_chat(messages)
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')

In [8]:
generate_kwargs = dict(
    inputs,
    max_new_tokens=1024,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
)
r = model.generate(**generate_kwargs)
print(tokenizer.decode(r[0]))

<s> [INST] <<SYS>>
awak adalah AI yang mampu jawab segala soalan
<</SYS>>

awat malaysia ada jabatan koko, malaysia bukan buat keluaq koko banyak pun [/INST] Penanaman koko di Malaysia bermula pada tahun 1950-an, dengan pengeluaran koko kering pertama dihasilkan pada tahun 1958. Pada masa itu, negara ini menjadi pembekal koko terbesar di dunia, dan industri ini terus berkembang dan mewujudkan ribuan pekerjaan dalam pemprosesan dan pengeluaran koko.

Walau bagaimanapun, penurunan harga koko di pasaran antarabangsa pada tahun 1980-an menyebabkan pengeluaran koko Malaysia menurun, dan negara ini telah mengubah tumpuan daripada penanaman koko kepada komoditi lain seperti minyak kelapa sawit dan getah.

Walaupun Malaysia bukan pengeluar koko yang besar, industri koko tempatan masih penting dari segi ekonomi dan sosial. Koko ialah komoditi yang diperdagangkan di bursa tempatan, dan pengeluar koko tempatan menghasilkan koko kering yang dieksport ke seluruh dunia.

Jabatan Koko Malaysia ialah 